# [MCB 32]: Lab 04 - Muscle Excitability and Contractility
 
---

### Professor Robin Ball

We will introduce you to data analysis using Python and Jupyter notebooks using the data you collect in this lab. You will receive more information about this during lab.

*Estimated Time: ~1 Hour*

---

### Table of Contents

[Intro to Jupyter notebooks](#section 0)<br>

1.[Introduction to Python](#section 1)<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a   [Entering and Naming your data](#section 1a)<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; b   [Graphing your data](#section 1b)<br>


## Welcome to Jupyter  <a id='section 0'></a>

Welcome to the Jupyter Notebook! **Notebooks** are documents that can contain text, code, visualizations, and more. We'll be using them in this lab to manipulate and visualize our data.

A notebook is composed of rectangular sections called **cells**. There are 2 kinds of cells: markdown and code. A **markdown cell**, such as this one, contains text. A **code cell** contains code in Python, a programming language that we will be using for the remainder of this module. You can select any cell by clicking it once. After a cell is selected, you can navigate the notebook using the up and down arrow keys.

To run a code cell once it's been selected, 
- press Shift-Enter, or
- click the Run button in the toolbar at the top of the screen. 

If a code cell is running, you will see an asterisk (\*) appear in the square brackets to the left of the cell. Once the cell has finished running, a number will replace the asterisk and any output from the code will appear under the cell.

In [ ]:
# run this cell
print("Hello World!")

You'll notice that many code cells contain lines of blue text that start with a `#`. These are *comments*. Comments often contain helpful information about what the code does or what you are supposed to do in the cell. The leading `#` tells the computer to ignore them.

In [ ]:
# this is a comment- running the cell will do nothing!

Code cells can be edited any time after they are highlighted. Try editing the next code cell to print your name.

In [ ]:
# edit the code to print your name
print("Hello: my name is NAME")

#### Saving and Loading

Your notebook can record all of your text and code edits, as well as any graphs you generate or calculations you make. You can save the notebook in its current state by clicking Control-S, clicking the floppy disc icon in the toolbar at the top of the page, or by going to the File menu and selecting "Save and Checkpoint".

The next time you open the notebook, it will look the same as when you last saved it.

**Note:** after loading a notebook you will see all the outputs (graphs, computations, etc) from your last session, but you won't be able to use any variables you assigned or functions you defined. You can get the functions and variables back by re-running the cells where they were defined- the easiest way is to highlight the cell where you left off work, then go to the Cell menu at the top of the screen and click "Run all above". You can also use this menu to run all cells in the notebook by clicking "Run all".

#### Completing the Notebooks


<div class="alert alert-info"> 

**QUESTION** cells are in blue and ask you to enter in lab data, make graphs, or do other lab tasks. To receive full credit for your lab, you must complete all **QUESTION** cells.


</div>



# 1. Python <a id='section 1'></a>

**Python** is  programming language- a way for us to communicate with the computer and give it instructions. Just like any language, Python has a *vocabulary* made up of words it can understand, and a *syntax* giving the rules for how to structure communication.

Python doesn't have a large vocabulary or syntax, but it can be used for many, many computational tasks.

Bits of communication in Python are called **expressions**- they tell the computer what to do with the data we give it.

Here's an example of an expression. 

In [ ]:
# an expression
14 + 20

When you run the cell, the computer evaluates the expression and prints the result. Note that only the last line in a code cell will be printed, unless you explicitly tell the computer you want to print the result.

In [ ]:
# more expressions. what gets printed and what doesn't?
100 / 10

print(4.3 + 10.98)

33 - 9 * (40000 + 1)

884

Many basic arithmetic operations are built in to Python, like `*` (multiplication), `+` (addition), `-` (subtraction), and `/` (division). There are many others, which you can find information about [here](http://www.inferentialthinking.com/chapters/03/1/expressions.html). 

The computer evaluates arithmetic according to the PEMDAS order of operations (just like you probably learned in middle school): anything in parentheses is done first, followed by exponents, then multiplication and division, and finally addition and subtraction.

In [ ]:
# before you run this cell, can you say what it should print?
4 - 2 * (1 + 6 / 3)

#### A Note on Errors <a id="subsection error"></a>

Python is a language, and like natural human languages, it has rules.  It differs from natural language in two important ways:
1. The rules are *simple*.  You can learn most of them in a few weeks and gain reasonable proficiency with the language in a semester.
2. The rules are *rigid*.  If you're proficient in a natural language, you can understand a non-proficient speaker, glossing over small mistakes.  A computer running Python code is not smart enough to do that.

Whenever you write code, you'll make mistakes.  When you run a code cell that has errors, Python will sometimes produce error messages to tell you what you did wrong.

Errors are normal; experienced programmers see errors all the time.  When you make an error, you just have to find the source of the problem, fix it, and move on.

We have made an error in the next cell.  Delete the `#`, then run it and see what happens.


In [ ]:
# print("This line is missing something."

You should see something like this (minus our annotations):

<img src="img/error.jpg"/>

The last line of the error output attempts to tell you what went wrong.  The *syntax* of a language is its structure, and this `SyntaxError` tells you that you have created an illegal structure.  "`EOF`" means "end of file," so the message is saying Python expected you to write something more (in this case, a right parenthesis) before finishing the cell.

There's a lot of terminology in programming languages, but you don't need to know it all in order to program effectively. If you see a cryptic message like this, you can often get by without deciphering it.  (Of course, if you're frustrated, you can usually find out by searching for the error message online or asking course staff for help).

### 1a. Entering and Naming your data <a id='section 1a'></a>
Sometimes, the values you work with can get cumbersome- maybe the expression that gives the value is very complicated, or maybe the value itself is long. In these cases it's useful to give the value a **name**.

We can name values using what's called an *assignment* statement.

In [ ]:
# assigns 442 to x
x = 442

The assignment statement has three parts. On the left is the *name* (`x`). On the right is the *value* (442). The *equals sign* in the middle tells the computer to assign the value to the name.

You'll notice that when you run the cell with the assignment, it doesn't print anything. But, if we try to access `x` again in the future, it will have the value we assigned it.

In [ ]:
# show the value of x
x

You can also assign names to expressions. The computer will compute the expression and assign the name to the result of the computation.

In [ ]:
y = 50 * 2 + 1
y

We can then use these names as if they were whatever they stand for (in this case, numbers).

In [ ]:
x - 42

In [ ]:
x + y

#### Lists
In Python, you can also make lists of numbers. A Python **list** is enclosed in square brackets. Items inside the list are separated by commas.

In [ ]:
# a list
[7.0, 6.24, 9.98, 4]

Lists can have names too, which is handy for when you want to want to save a set of items without writing them out over and over again.

In [ ]:
my_list = [4, 8, 15, 16, 23, 42]
my_list

<div class="alert alert-info">

**QUESTION**: Make four lists. The lists should be as follows: 

</div>

- `stim_amp1` = a list of the average *stimulus amplitude* for *subject 1's* five twitches
- `twitch_dur1` = a list of the average *twitch duration* for *subject 1's* five twitches
- `stim_amp2` = a list of the average *stimulus amplitude* for *subject 2's* five twitches
- `twitch_dur2` = a list of the average *twitch duration* for *subject 2's* five twitches

Hint: make sure the items in each list are entered in order, and that there are exactly five items per list (because you recorded the twitches at five different stimulation amplitudes).

In [ ]:
# stimulus amplitudes, subject 1
stim_amp1 = ...

#twitch durations, subject 1
twitch_dur1 = ...

# stimulus amplitudes, subject 2
stim_amp2 = ...

#twitch durations, subject 2
twitch_dur2 = ...

### 1b. Graphing your Data <a id='section 1b'></a>
We've seen that values can have names (often called **variables**), but operations may also have names. A named operation is called a **function**. Python has some functions built into it.

In [ ]:
# a built-in function 
round

Functions get used in *call expressions*. A call expression is a command to the computer telling it to use a certain function. Call expressions have three parts:
1. the name of the function
2. a set of parentheses to the right of the function
3. a value or values within the parentheses for the function to act upon

For example, the `round` function returns the number it was given, rounded to the nearest whole number. The data it gets is a number that will get rounded.

In [ ]:
# a call expression using round
round(1988.74699)

A function may also be called on more than one value (called *arguments*). For instance, the `min` function takes however many arguments you'd like and returns the smallest. Multiple arguments are separated by commas.

Try adding arguments to the `min` function in the cell below. What do you think `min` does?

In [ ]:
# a call expression using min
min(1, 2)

Python has a lot of [built-in functions](https://docs.python.org/3/library/functions.html) (that is, functions that are already named and defined in Python), but even more functions are stored in collections called *modules*. 

Let's load a graphing module to plot our data. We'll name our module `plt`. Run the next two cells to first download then load the module.

In [ ]:
!pip install matplotlib

In [ ]:
# This line load the module
import matplotlib.pyplot as plt

# These lines set some formatting settings so our graphs look nicer
% matplotlib inline
plt.style.use('fivethirtyeight')

We're interested in a function called `plot`, which makes line graphs. We can tell the computer to find `plot` in the `plt` module using what's called *dot notation*:

In [ ]:
# use the plot function
plt.plot()

`plot` needs to get data to plot as its arguments. In this case, we can give it the lists of data we made earlier. The data goes inside the parentheses: the data that goes on the x-axis is the first argument, and the data that goes on the y-axis is the second.

Here's an example of how to plot Stimulus Amplitude against Stimulus Duration for Subject 1. Stimulus Amplitude is on the x-axis, so it goes first in the parentheses.

In [ ]:
# create a line plot of data for subject 1
plt.plot(stim_amp1, twitch_dur1)

# a function to show the line plot
plt.show()

As good scientists, we know that this graph is missing a lot. Our axes, graph, and line should all be labeled. Fortunately, the `plt` module has functions to do all of those things. 

Read over the call expressions in the next cell to figure out what each does, then run the cell to make the graph.

<div class="alert alert-warning">

Note that we put labels and titles in quotation marks. This tells the computer that they are text data (also called *strings*). If we didn't put them in quotes, the computer would think they are variables, then throw a fit when it couldn't find what they stood for.

</div>

In [ ]:
# create the line plot for subject 1
plt.plot(stim_amp1, twitch_dur1, label="Subject 1")

# label the x-axis
plt.xlabel("Stimulus amplitude (mA)")

# label the y-axis
plt.ylabel("Twitch duration (sec)")

# title the graph
plt.title("Twitch size vs. stimulus strength")

# show the legend
plt.legend()

# show the plot
plt.show()

<div class="alert alert-info">

**QUESTION**: Make a graph of the amplitude of the twitch (i.e. twitch tension) versus the stimulus amplitude (x-axis). Label your axes. Include lines for both subjects (and label them).

</div>

Hint: You need to replace every `...` with a variable name or a text label. Remember that text labels MUST go inside quotes. *You will need to create two new lists for the twitch tension (amplitude) for subject 1 and 2.* If you look at the previous example, you'll see how to do most of it!

In [ ]:
# plot the line for subject 1
plt.plot(..., ..., label=...)

# plot the line for subject 2
plt.plot(..., ..., label=...)

# label the x-axis
plt.xlabel(...)

#label the y-axis
plt.ylabel(...)

# show the legend
plt.legend()

# show the plot
plt.show()

### Saving the Notebook as a PDF

Congrats on finishing your first lab notebook! To turn in this lab assignment follow the steps below:

>1. Click on `File` in the toolbar above.
2. Scroll down to `Download as`.
3. Click on PDF via LaTeX (.pdf)

Your lab instructor will explain to you what to do afterwards.


---



#### References

- Sections of "Intro to Jupyter", adapted from materials by Kelly Chen and Ashley Chien in [UC Berkeley Data Science Modules core resources](http://github.com/ds-modules/core-resources)
- "A Note on Errors" subsection and "error" image adapted from materials by Chris Hench and Mariah Rogers for the Medieval Studies 250: Text Analysis for Graduate Medievalists [data science module](https://github.com/ds-modules/MEDST-250).
- "Intro to Jupyter" and "Intro to Python" adapted from materials by Keeley Takimoto for the Berkeley Executive Education [Program on Data Science and Analytics Module](https://github.com/ds-modules/BEE)

---
Notebook developed by: Alex Nakagawa

Data Science Modules: http://data.berkeley.edu/education/modules
